In [2]:
import base64
import anthropic
from dotenv import load_dotenv
load_dotenv()
client = anthropic.Anthropic()

In [3]:
def read_file_and_encode(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_base64 = base64.standard_b64encode(pdf_file.read()).decode("utf-8")
    
    return pdf_base64

In [23]:
def sections_1_2_instructions():
    return """
    ## Core Requirements
        - Use ONLY information from the provided V&V document - no external knowledge or invented details
        - Output a coherent, text-only narrative report with clear section headings
        - If information is missing, omit that section or note it as unavailable
        - Focus on stability-related content: look for keywords like "stability testing," "aging," "shelf life," "long-term performance"
    
    ### 1. Summary (Executive Summary)
        - Brief overview of study purpose and key findings
        - Critical data points from stability testing
        - Overall conclusion on product stability status and recommended shelf life

    ### 2. Introduction
        - Purpose and rationale of the stability study
        - Product description and context from V&V
        - Scope: duration, conditions, aspects evaluated, sample sizes
        - Regulatory/standard framework if mentioned
        - Key assumptions about test conditions
        

"""

In [ ]:
def get_system_instructions():
    return """
    ## Core Requirements
    - Use ONLY information from the provided V&V document - no external knowledge or invented details
    - Output a coherent, text-only narrative report with clear section headings
    - If information is missing, omit that section or note it as unavailable
    - Focus on stability-related content: look for keywords like "stability testing," "aging," "shelf life," "long-term performance"

    ## Report Structure
    Generate the following sections in order:

    

    ### 3. Stability Study Methodology
    - Study design overview: what was tested, how long, under what conditions
    - Materials and samples: batch numbers, sample sizes, packaging
    - Test parameters and measurement methods
    - Acceptance criteria and pass/fail standards
    - Any deviations or protocol changes noted

    ### 4. Results
    - Present data chronologically or by parameter as organized in V&V
    - Include all relevant numerical data and observations in narrative form
    - Note trends and changes over time
    - Report compliance with acceptance criteria
    - Include visual/physical observations
    - State facts only - no interpretation here

    ### 5. Discussion
    - Analyze overall trends and their significance
    - Compare results to acceptance criteria
    - Explain implications for product quality and real-world use
    - Acknowledge study limitations
    - Provide recommendations based on findings
    - Synthesize rather than repeat raw data

    ### 6. Conclusion and Recommendations
    - Clear statement on overall product stability
    - Specific shelf life and storage condition recommendations
    - Summary of compliance with all criteria
    - Key recommendations for labeling, handling, further testing, or process changes
    - Confident closing statement

    ## Writing Guidelines
    - Write in complete sentences with clear, professional language
    - Organize content logically with smooth transitions between sections
    - Include specific data points with units when available
    - Use comparative language for trends (e.g., "remained constant," "showed slight decline")
    - Maintain objectivity and base all statements on V&V evidence
    - Ensure the report reads as a standalone document

    Generate a thorough stability report that comprehensively converts the V&V data into the standard stability report format.
    """

In [ ]:


def get_anthropic_response(pdf_base64, query,system_instructions):
    client = anthropic.Anthropic()

    with client.messages.stream(
        model="claude-opus-4-20250514",
        max_tokens=16000,
        system=system_instructions,
        thinking={
            "type": "enabled",
            "budget_tokens": 10000
        },
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "document",
                        "source": {
                            "type": "base64",
                            "media_type": "application/pdf",
                            "data": pdf_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": query
                    }
                ]
            }
        ],
    ) as stream:
        content = []
        for event in stream:
            if hasattr(event, 'delta') and hasattr(event.delta, 'text'):
                content.append(event.delta.text)

        return ''.join(content)




In [ ]:
pdf_data = read_file_and_encode('docs/PRJ-0695-Silicon Change on NIP valve-Design Verification & Validation Report.pdf')
section_1_2_response=get_anthropic_response(pdf_data, "Generate a product stability report from the V&V document.",sections_1_2_instructions())

In [ ]:
section_1_2_response.split('\n\n')

['# Product Stability Report\n## Silicon Change on NIP Valve - Design Verification & Validation\n### Document: Doc-331335, Project ID: 0695',
 '## Summary (Executive Summary)',
 'This stability study evaluated the NIP Stand-Alone valve (PN 220808L) with new silicone septum material (SILPURAN® 6000/50 A+B, PN 590422) to establish shelf life following a material change from discontinued TSL097 Silicone QP1 50 LSR. Critical stability testing included accelerated aging at 52°C and 60°C for periods simulating up to 5 years, combined with sterilization exposure of up to 42.8 kGy gamma irradiation plus two ETO cycles. All functional performance tests passed successfully after 5-year equivalent aging, supporting a recommended shelf life of five years for products containing the new silicone valve material.',
 '## Introduction',
 "The stability study was initiated due to Dow Corning's discontinuation of TSL097 Silicone QP1 50 LSR raw material, necessitating replacement with SILPURAN® 6000/50 A+

In [4]:
def section_3_instructions():
    return """
read the reference table at section 3 of the V&V document and a table with only the relevant refecnce data for section 3 of the stability report porotocol.
    """

In [5]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

file = client.files.create(
    file=open("docs/PRJ-0695-Silicon Change on NIP valve-Design Verification & Validation Report.pdf", "rb"),
    purpose="user_data"
)

completion = client.chat.completions.create(
    model="o3",
    reasoning_effort="high",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    }
                },
                {
                    "type": "text",
                    "text": section_3_instructions(),
                },
            ]
        }
    ]
)

print(completion.choices[0].message.content)

Below is the information requested in two parts:

A. Complete reference list that appears in Section 3 of the “Design Verification & Validation Report – Silicon Change on NIP Valve” (pages 2-3).

B. A condensed table that keeps ONLY those references that are directly useful for writing, executing or supporting Section 3 (References) of the “Product Stability Test Protocol – PRJ-0695-Silicon Change on NIP valve” (Doc-332257).  
The filtering logic was: keep documents that (1) define or govern aging / shelf-life work, (2) document sterilisation history that may affect stability, or (3) capture stability/aging test results.



──────────────────────────────────────────────────  
A. Reference table in the V&V report (Section 3)

| # | Description / Title (as written) | Doc-ID | Rev. |
|---|----------------------------------|--------|------|
| 3.1 | MRD – Marketing Requirements Document | – | – |
| 3.2 | PRD – Product Requirements Document | Doc-310149 | B |
| 3.3 | Product risk-management 

In [15]:
def get_openai_response(file, query):
    client = OpenAI()

    completion = client.chat.completions.create(
    model="o3",
    reasoning_effort="high",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    }
                },
                {
                    "type": "text",
                    "text": query,
                },
            ]
        }
    ]
)

    return completion.choices[0].message.content

In [20]:
def load_file_to_openai(file_path):
    client = OpenAI()
    file = client.files.create(
        file=open(file_path, "rb"),
        purpose="user_data"
    )
    return file

In [21]:
file = load_file_to_openai("docs/PRJ-0695-Silicon Change on NIP valve-Design Verification & Validation Report.pdf")

In [22]:
file

FileObject(id='file-S9XhiDCjcrkNSuvCpbgF6n', bytes=741819, created_at=1749217915, filename='PRJ-0695-Silicon Change on NIP valve-Design Verification & Validation Report.pdf', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None)

In [23]:
response=get_openai_response(file, section_3_instructions())
print(response)

Below is

1.  A transcription of the complete “Section 3 – References” table that appears in the Design Verification & Validation Report “Silicon Change on NIP valve” (Doc-331335, Rev A, page 2).

2.  A condensed table that keeps **only the items that would normally be cited in Section 3 of the Product-Stability Test Protocol (Doc-332257, Rev A)**—i.e., documents that are either:
    • themselves the stability protocol, or  
    • supply data/results that are fed into the stability study (aging, sterilization, dimensional or laboratory results, manufacturing batch records, risk inputs, etc.).

--------------------------------------------------------------------
1.  FULL REFERENCE TABLE – V&V REPORT (page 2)
--------------------------------------------------------------------
| # | Description                                                                                                                         | Doc-ID      | Rev |
|---|-------------------------------------------------